In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

%cd /content/drive/MyDrive/CoLab/

In [ ]:
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')

import sys
sys.path.insert(0, "../")

import pandas as pd 
import numpy as np
import torch
from torchsummary import summary

torch.cuda.is_available()

Mounted at /content/drive


/content/drive/MyDrive/CoLab
